In [38]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

import numpy as np

In [39]:
df_recipes = pd.read_csv("recipes_info.csv")

In [21]:
# df_recipes.drop_duplicates(subset=['recipe_id'], keep="first", inplace=True)

In [40]:
#instantiate both packages to use
# encoder = OrdinalEncoder()
imputer = KNNImputer()
# imputer = IterativeImputer(KNeighborsRegressor())
# create a list of categorical columns to iterate over
cat_cols = [
    "country",
    "country_secondary",
    "dish_category",
    "dish_type",
    "diet_type",
    "carbohydrate_base",
    "carbohydrate_category",
    "protein",
    "protein_cut",
    "protein_type",
    "family_friendly",
    "spice_level",
]

encoders = {
    
}

def encode(data, col):
    '''function to encode non-null data and replace it in the original data'''
    
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    # print(impute_reshape)
    encoders[col].fit(impute_reshape)
    impute_ordinal = encoders[col].transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for col in cat_cols:
    encoders[col] = OrdinalEncoder()
    encode(df_recipes[col], col)

C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [41]:
df_temp = pd.DataFrame(np.round(imputer.fit_transform(df_recipes[cat_cols])),columns = cat_cols)

In [42]:
def decode(data, col):
    '''function to encode non-null data and replace it in the original data'''
    
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for decoding
    impute_reshape = nonulls.reshape(-1,1)
    #decode date
    # print(impute_reshape)
    impute_ordinal = encoders[col].inverse_transform(impute_reshape)
    # print(impute_ordinal)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for col in cat_cols:
    decode(df_temp[col], col)

C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\gkanchev\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [44]:
dif_list = list(set(df_recipes.columns.values) - set(df_temp.columns.values))

for col in dif_list:
    df_temp[col] = df_recipes[col]

In [98]:
df_temp.to_csv("recipes_imputed_knn.csv", index=False)